In [24]:
import os
import joblib
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [25]:
data, labels, freq = joblib.load('/home/lucas_nunes/Documentos/datasets/lofar_data/lofar_data_file_fft_1024_overlap_0_decimation_3_spectrum_left_400.npy')
runs_per_class = joblib.load('/home/lucas_nunes/Documentos/datasets/runs_info/runs_info_1024_fft_pts_3_decimation.joblib')

In [26]:
print(data.shape, labels.shape, freq.shape)
runs_per_class

(25844, 400) (25844,) (400,)


{'A': [range(0, 863),
  range(863, 1726),
  range(1726, 2587),
  range(2587, 3453),
  range(3453, 4312)],
 'B': [range(4312, 5501),
  range(5501, 6257),
  range(6257, 7246),
  range(7246, 8556),
  range(8556, 9724),
  range(9724, 10837),
  range(10837, 11864),
  range(11864, 12767),
  range(12767, 13217),
  range(13217, 14093)],
 'C': [range(14093, 14590),
  range(14590, 15013),
  range(15013, 15262),
  range(15262, 15689),
  range(15689, 16117),
  range(16117, 16440),
  range(16440, 16703),
  range(16703, 17061),
  range(17061, 17926)],
 'D': [range(17926, 18572),
  range(18572, 19289),
  range(19289, 20148),
  range(20148, 20794),
  range(20794, 21855),
  range(21855, 22431),
  range(22431, 23168),
  range(23168, 24215),
  range(24215, 25200),
  range(25200, 25844)]}

In [27]:
from sklearn.preprocessing import normalize
data = normalize(data)

In [28]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca_data = pca.fit_transform(data)

In [29]:
from sklearn.model_selection import train_test_split
#Getting the data for the classifier for differentiating the class1 from the others
binary_label = np.where(labels == 1, 1, -1)
x_train, x_test, y_train, y_test = train_test_split(pca_data, labels, test_size = 0.1, shuffle=True)

In [30]:
from sklearn.svm import SVC
svm_model = SVC(verbose=True)
svm_model.fit(x_train, y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [31]:
predictions = svm_model.predict(x_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[375  28  19  14]
 [ 10 905  46  16]
 [ 12  87 270  13]
 [ 13  20   9 748]]
              precision    recall  f1-score   support

           0       0.91      0.86      0.89       436
           1       0.87      0.93      0.90       977
           2       0.78      0.71      0.74       382
           3       0.95      0.95      0.95       790

    accuracy                           0.89      2585
   macro avg       0.88      0.86      0.87      2585
weighted avg       0.89      0.89      0.89      2585



In [32]:
svm_binary  = SVC(verbose=True)
bin_y_train = np.where(y_train ==1, 1, -1)
bin_y_test = np.where(y_test ==1, 1, -1)
svm_binary.fit(x_train, bin_y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [33]:
predictions = svm_binary.predict(x_test)
print(predictions.shape)
print(predictions)
print(confusion_matrix(bin_y_test,predictions))
print(classification_report(bin_y_test,predictions))

(2585,)
[ 1 -1 -1 ... -1  1  1]
[[1495  113]
 [ 112  865]]
              precision    recall  f1-score   support

          -1       0.93      0.93      0.93      1608
           1       0.88      0.89      0.88       977

    accuracy                           0.91      2585
   macro avg       0.91      0.91      0.91      2585
weighted avg       0.91      0.91      0.91      2585



In [34]:
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(SVC())
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
print(predictions.shape)
print(predictions)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

(2585,)
[1 2 2 ... 3 1 1]
[[378  23  21  14]
 [  8 907  44  18]
 [ 16  93 258  15]
 [ 10  16   8 756]]
              precision    recall  f1-score   support

           0       0.92      0.87      0.89       436
           1       0.87      0.93      0.90       977
           2       0.78      0.68      0.72       382
           3       0.94      0.96      0.95       790

    accuracy                           0.89      2585
   macro avg       0.88      0.86      0.87      2585
weighted avg       0.89      0.89      0.89      2585



In [35]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.svm import SVC

class OneVsRestSVCCommittee():

    def __init__(self, class_mappping, expert_params=None):
        self.class_mapping = class_mappping
        if expert_params is None:
            expert_params = dict.fromkeys(list(class_mapping.keys()), dict())
        self.classifiers = {class_: SVC(**params) for class_, params in expert_params.items()}

    def fit(self, X, y, verbose=True):

        for class_name, class_value in self.class_mapping.items():
            if verbose:
                print(f'Fitting svm expert for class {class_name}')
            self.classifiers[class_name].fit(X, np.where(y == class_value, 1, -1))
    
    def get_params(self):
        return {class_: classifier.get_params() for class_, classifier in self.classifiers.items()}

    def predict(self, X):
        return np.column_stack([classifier.predict(X) for classifier in self.classifiers.values()])
    
    def score(self, X, y):
        stack = list()
        for class_name, class_value in self.class_mapping.items():
            stack.append(self.classifiers[class_name].score(X, np.where(y == class_value, 1, -1)))
        all_scores = np.column_stack(stack)
        del stack
        return all_scores   

    def evaluate(self, X, y, metrics):
        columns = np.array(self.class_mapping.keys())
        index = list()
        data = list()
        for metric in metrics:
            index.append(metric.__name__)
            data.append([metric(np.where(y == class_value, 1, -1), self.classifiers[class_name].predict(X))
                            for class_name, class_value in self.class_mapping.items()])
        
        frame = pd.DataFrame(data, index=index, columns=columns)
        return frame

    def set_params(self, class_, **params):
        self.classifiers[class_].set_params(**params)


In [36]:
classes_names = np.array(['A', 'B', 'C', 'D'])
classes_values = np.arange(4)
class_mapping = {class_: value for class_, value in zip(classes_names, classes_values)}

In [37]:
svc_committee = OneVsRestSVCCommittee(class_mapping)

In [38]:
svc_committee.fit(x_train, y_train)

Fitting svm expert for class A
Fitting svm expert for class B
Fitting svm expert for class C
Fitting svm expert for class D


In [39]:
predictions = np.argmax(svc_committee.predict(x_test), axis=1)
print(predictions.shape)
print(predictions)

(2585,)
[1 0 2 ... 3 1 1]


In [40]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[404  16   9   7]
 [ 81 863  23  10]
 [ 63  77 236   6]
 [ 35  15   6 734]]
              precision    recall  f1-score   support

           0       0.69      0.93      0.79       436
           1       0.89      0.88      0.89       977
           2       0.86      0.62      0.72       382
           3       0.97      0.93      0.95       790

    accuracy                           0.87      2585
   macro avg       0.85      0.84      0.84      2585
weighted avg       0.88      0.87      0.86      2585

